In [ ]:
import plotting as aplt
import control_experiment_1 as experiment_runner
from pathlib import Path
import numpy as np
import proplot as pplt
import xarray as xr
import pandas as pd
from plotting import OIFSPreprocessor, NEMOPreprocessor

In [ ]:
oifs_preprocessor = OIFSPreprocessor(
    np.datetime64("2014-07-01"), np.timedelta64(-7, "h")
)
nemo_preprocessor = NEMOPreprocessor(np.timedelta64(-7, "h"))

In [ ]:
plotting_output_directory = Path("plots/control_experiment_1")
plotting_output_directory.mkdir(exist_ok=True)

In [ ]:
naive_exp_ids = ["C1N0", "C1N1", "C1N2"]
schwarz_exp_ids = ["C1S0", "C1S1", "C1S2"]
max_schwarz_iters = experiment_runner.max_iters
setup = "PAPA"

In [ ]:
oifs_progvars_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/progvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
oifs_diagvars_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/diagvar.nc", preprocess=oifs_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]
nemo_t_grids_naive = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}/{exp_id}*_T.nc", preprocess=nemo_preprocessor.preprocess
    )
    for exp_id in naive_exp_ids
]

oifs_progvars_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/progvar.nc",
        preprocess=oifs_preprocessor.preprocess,
    )
    for exp_id in schwarz_exp_ids
]
oifs_diagvars_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/diagvar.nc",
        preprocess=oifs_preprocessor.preprocess,
    )
    for exp_id in schwarz_exp_ids
]
nemo_t_grids_cvg_swz = [
    xr.open_mfdataset(
        f"{setup}/{exp_id}_{max_schwarz_iters}/{exp_id}*_T.nc",
        preprocess=nemo_preprocessor.preprocess,
    )
    for exp_id in schwarz_exp_ids
]

exp_id = schwarz_exp_ids[0]
oifs_progvars_add_swz = []
oifs_diagvars_add_swz = []
nemo_t_grids_add_swz = []
for iter in range(1, max_schwarz_iters + 1):
    oifs_progvars_add_swz.append(
        xr.open_mfdataset(
            f"{setup}/{exp_id}_{iter}/progvar.nc",
            preprocess=oifs_preprocessor.preprocess,
        )
    )
    oifs_diagvars_add_swz.append(
        xr.open_mfdataset(
            f"{setup}/{exp_id}_{iter}/diagvar.nc",
            preprocess=oifs_preprocessor.preprocess,
        )
    )
    nemo_t_grids_add_swz.append(
        xr.open_mfdataset(
            f"{setup}/{exp_id}_{iter}/{exp_id}*_T.nc",
            preprocess=nemo_preprocessor.preprocess,
        )
    )

In [ ]:
colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 1
linestyles = ["--", ":", "-."]

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(
    f"{naive_exp_ids[0]}, {naive_exp_ids[1]}, {naive_exp_ids[2]}", y=0.95, size=14
)


aplt.create_atm_temps_plot(
    axs[0], oifs_progvars_naive, colors, alpha, labels, linestyles
)
aplt.create_oce_ssts_plot(axs[1], nemo_t_grids_naive, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(
    axs[2], oifs_diagvars_naive, colors, alpha, labels, linestyles
)
fig.savefig(
    plotting_output_directory / "naive_coupling_schemes.pdf",
    bbox_inches="tight",
)

In [ ]:
colors = ["m", "c", "y"]
labels = ["parallel", "atm-first", "oce-first"]
alpha = 0.7
linestyles = ["--", ":", "-."]

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(f"Final Schwarz Iterations for Control Experiment 1", y=0.95, size=14)

aplt.create_atm_temps_plot(
    axs[0], oifs_progvars_cvg_swz, colors, alpha, labels, linestyles
)
aplt.create_oce_ssts_plot(
    axs[1], nemo_t_grids_cvg_swz, colors, alpha, labels, linestyles
)
aplt.create_atm_ssws_plot(
    axs[2], oifs_diagvars_cvg_swz, colors, alpha, labels, linestyles
)
fig.savefig(
    plotting_output_directory / "schwarz_coupling.pdf",
    bbox_inches="tight",
)

# Naive vs. Schwarz Solution

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvars_cvg_swz[0]]
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvars_cvg_swz[0]]
nemo_t_grids = [*nemo_t_grids_naive, nemo_t_grids_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atmosphere-first", "ocean-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=3, ncols=1, spany=False)
fig.set_size_inches(15, 10)
fig.suptitle(f"Non-iterative vs. Schwarz (CE1)", y=0.95, size=14)

aplt.create_atm_temps_plot(axs[0], oifs_progvars, colors, alpha, labels, linestyles)
aplt.create_oce_ssts_plot(axs[1], nemo_t_grids, colors, alpha, labels, linestyles)
aplt.create_atm_ssws_plot(axs[2], oifs_diagvars, colors, alpha, labels, linestyles)
axs[0].legend(ncols=1)
fig.savefig(
    plotting_output_directory / "coupling_comparison.pdf",
    bbox_inches="tight",
)

# Boundary Layer Type

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvars_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
markers = [".", ".", ".", "."]
shifts = [-0.06, -0.02, 0.02, 0.06]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Boundary Layer Type in Control Experiment 1", y=0.95, size=14)

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    marker = markers[i]
    shift = shifts[i]
    ax.plot(
        oifs_diagvar.pbl_type + shift,
        color=color,
        label=label,
        alpha=alpha,
        ls="none",
        marker=marker,
    )
ax.set_ylabel("Boundary layer type")
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(["DS", "DC", "Sc", "Cu"])
ax.legend(ncol=1)
fig.savefig(
    plotting_output_directory / "pbl_type_comparison.pdf",
    bbox_inches="tight",
)

# Boundary Layer Heights

## Atmosphere

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvars_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Boundary Layer Height (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.pbl_height,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylabel(r"Boundary Layer Height $[m]$")
ax.set_xlabel("Time")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "pbl_height.pdf",
    bbox_inches="tight",
)

In [ ]:
fig, ax = pplt.subplots()
fig.set_size_inches(8, 5)

oifs_diagvar = oifs_diagvars_cvg_swz[0]
ax.plot(oifs_diagvar.height_f[:, 50], color="k")
fig.suptitle("Height at Atmospheric Model Level 50")
ax.set_title("")
ax.set_xlabel("Time")
fig.savefig(
    plotting_output_directory / "height_L50.pdf",
    bbox_inches="tight",
)

$\Rightarrow$ for a vertical profile of the atmospheric boundary layer, the last 10 atmospheric levels should suffice!

## Ocean

In [ ]:
nemo_t_grids = [*nemo_t_grids_naive, nemo_t_grids_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Mixed Layer Depth (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    nemo_t_grid = nemo_t_grids[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        nemo_t_grid.somxl010[:, 0, 0],
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.set_ylim([0, 10])
ax.set_ylabel(r"Mixed Layer Depth $[m]$")
ax.set_xlabel("Time")
ax.set_title("")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "mldepth.pdf",
    bbox_inches="tight",
)

According to [this report, p. 18](https://www.drakkar-ocean.eu/publications/reports/orca025-grd100-report-dussin), the first 10 levels should suffice for NEMO as well.

# Boundary Layer Type across Iterations

How often does the PBL type change for a particular time step compared to the previous iteration?

In [ ]:
pbl_type_changes = oifs_diagvars_add_swz[0].pbl_type.copy()
pbl_type_changes.data[:] = 0
pbl_types = [oifs_diagvar.pbl_type for oifs_diagvar in oifs_diagvars_add_swz]

previous_pbl_type = pbl_types[0]
for iter in range(1, max_schwarz_iters):
    pbl_type = pbl_types[iter]
    pbl_type_not_equal = np.where(pbl_type.data != previous_pbl_type.data)
    pbl_type_changes.data[pbl_type_not_equal] += 1
    previous_pbl_type = pbl_type

fig, ax = pplt.subplots()
fig.set_size_inches(8, 5)
fig.suptitle(f"Boundary Layer Type Change Counts (CE1)", size=14)
ax.plot(
    pbl_type_changes,
    color="k",
    alpha=1,
    ls="none",
    marker=".",
)
ax.set_ylabel("# of PBL type changes")
ax.set_title("+1 if boundary layer type changed between subsequent iterations")
fig.savefig(
    plotting_output_directory / "pbl_type_change_counter.pdf",
    bbox_inches="tight",
)

For the points which change the most frequently: Plot `PBL_Type` vs. iteration

In [ ]:
threshold = 3
frequent_changer_indices = (pbl_type_changes.to_numpy() > threshold).nonzero()[0]
fig, ax = pplt.subplots()
fig.set_size_inches(w=8, h=5)
x_data = np.arange(1, max_schwarz_iters + 1)
for frequent_changer_index in frequent_changer_indices:
    pbl_type_per_iteration = np.zeros(max_schwarz_iters)
    for iter in range(max_schwarz_iters):
        data = pbl_types[iter].data[frequent_changer_index]
        pbl_type_per_iteration[iter] = data
    ax.plot(
        x_data,
        pbl_type_per_iteration,
        label=frequent_changer_index,
        ls="none",
        marker=".",
        alpha=1,
    )
ax.format(title="", xlabel="Schwarz Iteration", ylabel="Boundary Layer Type")
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(["DS", "DC", "Sc", "Cu"])
ax.set_xticks(x_data[::2])
fig.suptitle(f"BL Type over iterations for points with > {threshold} changes")

In [ ]:
oifs_diagvars = oifs_diagvars_add_swz

max_changes_index = np.argmax(pbl_type_changes.to_numpy())
pbl_type_per_iteration = np.zeros(max_schwarz_iters)
ssh_flux_per_iteration = np.zeros(max_schwarz_iters)
slh_flux_per_iteration = np.zeros(max_schwarz_iters)
for iter in range(max_schwarz_iters):
    pbl_type_per_iteration[iter] = oifs_diagvars[iter].pbl_type.data[max_changes_index]
    ssh_flux_per_iteration[iter] = oifs_diagvars[iter].sfc_sen_flx.data[
        max_changes_index
    ]
    slh_flux_per_iteration[iter] = oifs_diagvars[iter].sfc_lat_flx.data[
        max_changes_index
    ]

x_data = np.arange(1, max_schwarz_iters + 1)
fig, axs = pplt.subplots(nrows=2, spany=False)
fig.set_size_inches(w=8, h=5)
ax = axs[0]
ax.plot(
    x_data,
    pbl_type_per_iteration,
    ls="none",
    marker=".",
    color="k",
)
ax.format(
    title="Boundary Layer Type",
    xlabel="Schwarz Iteration",
    ylabel="Boundary Layer Type",
    xminorlocator=1,
    xlocator=x_data[::2],
)
ax.set_xticks(x_data[::2])
ax.set_yticks([0, 1, 2, 3])
ax.set_yticklabels(["DS", "DC", "Sc", "Cu"])

ax = axs[1]
ax.plot(
    x_data,
    ssh_flux_per_iteration,
    ls="none",
    marker=".",
    label="Surface Sensible Heat Flux",
)
ax.plot(
    x_data,
    slh_flux_per_iteration,
    ls="none",
    marker=".",
    label="Surface Latent Heat Flux",
)
ax.format(
    title="Surface Heat Fluxes",
    xlabel="Schwarz Iteration",
    ylabel=r"Heat Flux $[W m^{-2}]$",
    xminorlocator=1,
    xlocator=x_data[::2],
)
ax.legend()

fig.savefig(
    plotting_output_directory / "pbl_type_heat_flx.pdf",
    bbox_inches="tight",
)

# Surface Heat Fluxes

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvars_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, axs = pplt.subplots(nrows=2, spany=False)
fig.set_size_inches(10, 8)
fig.suptitle(f"Surface Heat Fluxes in Control Experiment 1", y=0.95, size=14)

ax = axs[0]
for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_sen_flx,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.legend(ncols=1)
ax.format(ylabel=r"Sensible Heat Flux $[W m^{-2}]$", title="", xlabel="Time")

ax = axs[1]

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.sfc_lat_flx,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(ylabel=r"Latent Heat Flux $[W m^{-2}]$", title="", xlabel="Time")

fig.savefig(
    plotting_output_directory / "sfc_heat_flux_comparison.pdf",
    bbox_inches="tight",
)

# Cloud Cover

In [ ]:
oifs_diagvars = [*oifs_diagvars_naive, oifs_diagvars_cvg_swz[0]]

colors = ["m", "c", "y", "k"]
labels = ["parallel", "atm-first", "oce-first", "converged SWR"]
alpha = 1
linestyles = ["--", ":", "-.", "-"]

fig, ax = pplt.subplots()
fig.set_size_inches(10, 6)
fig.suptitle(f"Total Cloud Cover (Control Experiment 1)", y=0.95, size=14)

for i in range(len(colors)):
    oifs_diagvar = oifs_diagvars[i]
    color = colors[i]
    label = labels[i]
    linestyle = linestyles[i]
    ax.plot(
        oifs_diagvar.total_cloud,
        color=color,
        label=label,
        alpha=alpha,
        ls=linestyle,
    )
ax.format(title="", xlabel="Time", ylabel="Cloud Cover")
ax.legend(ncols=1)
fig.savefig(
    plotting_output_directory / "ccover.pdf",
    bbox_inches="tight",
)

# Vertical Profiles

## Atmosphere

### Vertical Temperature Profile

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvars_cvg_swz[0]]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Atmospheric Temperature Profiles in BL (Control Experiment 1)",
    y=1,
    size=14,
)

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.t[:, 50:] - 273.15,
        levels=10,
        vmin=0,
        vmax=15,
        transpose=True,
        discrete=True,
    )
    ax.format(xlabel="Time", ylabel="Air Pressure [hPa]", title=ax_title)
    ax.invert_yaxis()
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(
    plotting_output_directory / "air_temperature_profile.pdf",
    bbox_inches="tight",
)

### Vertical Humidity Profile

In [ ]:
oifs_progvars = [*oifs_progvars_naive, oifs_progvars_cvg_swz[0]]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Atmospheric Humidity Profiles in BL (Control Experiment 1)",
    y=1,
    size=14,
)

for i in range(len(oifs_progvars)):
    ax = axs[i]
    ax_title = titles[i]
    oifs_progvar = oifs_progvars[i]
    oifs_progvar = oifs_progvar.assign_coords(
        air_pressure=("nlev", oifs_progvar.pressure_f[0].data / 100)
    )
    oifs_progvar = oifs_progvar.swap_dims({"nlev": "air_pressure"})
    im = ax.contourf(
        oifs_progvar.q[:, 50:],
        vmin=0,
        vmax=0.01,
        transpose=True,
        discrete=True,
    )
    ax.set_ylabel("Air Pressure [hPa]")
    ax.invert_yaxis()
    ax.set_title(ax_title)
fig.colorbar(im, loc="b", title="Water Vapor Mixing Ratio [kg/kg]")
fig.savefig(
    plotting_output_directory / "q_profile.pdf",
    bbox_inches="tight",
)

## Ocean

### Vertical Temperature Profile

In [ ]:
nemo_t_grids = [*nemo_t_grids_naive, nemo_t_grids_cvg_swz[0]]

titles = [
    "Parallel",
    "Sequential Atmosphere-First",
    "Sequential Ocean-First",
    "Converged Schwarz Waveform Relaxation",
]

fig, axs = pplt.subplots(nrows=4, ncols=1)
fig.set_size_inches(15, 13)
fig.suptitle(
    f"Vertical Ocean Temperature Profiles in Mixed Layer (Control Experiment 1)",
    y=1,
    size=14,
)

for i in range(len(nemo_t_grids)):
    ax = axs[i]
    ax_title = titles[i]
    nemo_t_grid = nemo_t_grids[i]
    im = ax.contourf(
        nemo_t_grid.votemper[:, :10, 0, 0],
        levels=10,
        vmin=10,
        vmax=12,
        discrete=True,
        transpose=True,
    )
    ax.invert_yaxis()
    ax.format(title=ax_title, xlabel="Time", ylabel="Depth [m]")
cax = axs[-1]
fig.colorbar(im, loc="b", title="Temperature [°C]")
fig.savefig(
    plotting_output_directory / "oce_temperature_profiles.pdf",
    bbox_inches="tight",
)